In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Load the data and format it as a dataframe
cancer = load_breast_cancer(as_frame=True)

# print the first 5 rows of the data
display(cancer.data.head())

# training model
def model_training(model,train_data,train_target,test_data,test_target):
    model.fit(train_data,train_target)
    return model

# print the accuracy of the model
def model_evaluation(model,test_data,predict_target):
    print(f'Model Evaluation:{ model.score(test_data,predict_target)}\n')

# print classification report
def print_classification_report(model,test_data,test_target):
    predict_target = model.predict(test_data)
    print(f'Classification Report:\n{classification_report(y_true=test_target, y_pred=predict_target)}\n')
    

In [ ]:

# split the data into training and testing
cancer_data_train,cancer_data_test,cancer_target_train,cancer_target_test = train_test_split(cancer.data,cancer.target, test_size=0.2, random_state=42)

# model training
models = [SVC(),DecisionTreeClassifier(),LogisticRegression()]


# single model training
for model in models:
    print(f'Model: {model} \n')
    model_training(model=model, train_data=cancer_data_train, train_target=cancer_target_train, test_data=cancer_data_test, test_target=cancer_target_test)
    model_evaluation(model=model, test_data=cancer_data_test, predict_target=cancer_target_test)
    print_classification_report(model=model, test_data=cancer_data_test, test_target=cancer_target_test)
    

In [ ]:
# voting classifier, voting='hard' means majority voting
voting_model = VotingClassifier(estimators=[('svm',SVC()),('dt',DecisionTreeClassifier()),('lr',LogisticRegression())],voting='hard')
# stacking classifier, final_estimator is the final estimator which will be used to predict the output
stack_model = StackingClassifier(estimators=[('svm',SVC()),('ada',DecisionTreeClassifier()),('ext',LogisticRegression())],final_estimator=DecisionTreeClassifier())

ensemble_models = [voting_model,stack_model]
# ensemble model training
for model in ensemble_models:
    print('Model:',model)
    model_training(model=model, train_data=cancer_data_train, train_target=cancer_target_train, test_data=cancer_data_test, test_target=cancer_target_test)
    model_evaluation(model=model, test_data=cancer_data_test, predict_target=cancer_target_test)
    print_classification_report(model=model, test_data=cancer_data_test, test_target=cancer_target_test)